# Cycle GAN
source: https://arxiv.org/abs/1703.10593

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
import sys
from os.path import join
from fastai.vision import *

sys.path.insert(0, '/'.join(sys.path[0].split('/')[:-1] + ['scripts']))
from bleu_score import *

In [3]:
# i need to have better memory of torch funcs
# ??nn.ConvTranspose2d
# ??nn.ReLU
# ??nn.Conv2d

In [4]:
#export
def TConvNReLU(i, o, norm, k=3, s=2, b=True):
    '''Tranpose convolutional layer + normalization layer + ReLU activation.
        i: channel in
        o: channel out
        norm: normalization layer
        k: kernel size
        s: stride size
        b: whether to have bias or not in TransposeConv
    '''
    return [nn.ConvTranspose2d(i, o, k, s, padding=1, output_padding=1, bias=b),
            norm(o), 
            nn.ReLU(inplace=True)]

def PadConvNReLU(i, o, norm, pad_mode, k=3, s=1, p=1, b=True, act=True, init=nn.init.kaiming_normal_):
    '''PaddedConvolutional layer + normalization layer + ReLU activation.
        i: channel in
        o: channel out
        norm: normalization layer
        k: kernel size
        s: stride size
        p: padding size
        b: bias or not in TransposeConv
        act: whether to append ReLU after PadConvN
        init: initialization function
    '''
    layers = []
    #padding
    if pad_mode == 'reflection':
        layers.append(nn.ReflectionPad2d(p))
    elif pad_mode == 'border':
        layers.append(nn.ReplicationPad2d(p))
    #convolution
    conv = nn.Conv2d(i, o, k, s, p if pad_mode=='zeros' else 0, bias=b)
    if init:
        init(conv.weight)
        if hasattr(conv, 'bias') and hasattr(conv.bias, 'data'):
            conv.bias.data.fill_(0.)
    layers.append(conv)
    #normalization
    layers.append(norm(o))
    #activation
    if act:
        layers.append(nn.ReLU(inplace=True))
    return layers

In [5]:
#export
class ResnetBlock(nn.Module):
    def __init__(self, d, pad_mode, norm=None, dropout=0., b=True):
        '''ResBlock ( y = F(x) + x).
            pad_mode: padding mode (either "reflection" or "zero")
            norm: normalization layer
            dropout: dropout factor (0 to 1)
            b: whether to have bias or not in TransposeConv
        '''
        super().__init__()
        norm = norm if norm else nn.InstanceNorm2d
        layers = PadConvNReLU(d, d, norm, pad_mode, b=b)
        if dropout != 0: layers.append(nn.Dropout(dropout))
        layers += PadConvNReLU(d, d, norm, pad_mode, b=b, act=False)
        self.conv_block = nn.Sequential(*layers)
    
    def forward(self, x):
        return x + self.conv_block(x)

In [6]:
#export
def generator(i, o, c=64, norm=None, dropout=0., depth=6, pad_mode='reflection'):
    '''Generator of GAN (resnet based).
        i: channel in
        o: channel out
        c: start channel (changes are resnet is being built)
        dropout: dropout factor (0 to 1)
        depth: resnet depth
        pad_mode: padding mode (either "reflection" or "zero")
    '''
    norm = norm if norm else nn.InstanceNorm2d
    b = norm == nn.InstanceNorm2d
    layers = PadConvNReLU(i, c, norm=norm, pad_mode='reflection', k=7, p=3, b=b)
    for _ in range(2):
        layers += PadConvNReLU(c, c*2, norm=norm, pad_mode='zeros', s=2, b=b)
        c *= 2
    for _ in range(depth):
        layers.append(ResnetBlock(c, pad_mode, norm, dropout, b))
    for _ in range(2):
        layers += TConvNReLU(c, c//2, norm, b=b)
        c //= 2
    layers.append(nn.ReflectionPad2d(3))
    layers.append(nn.Conv2d(c, o, 7, padding=0))
    layers.append(nn.Tanh())
    return nn.Sequential(*layers)

In [7]:
res_gen = generator(3, 25)
res_gen

Sequential(
  (0): ReflectionPad2d((3, 3, 3, 3))
  (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
  (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (3): ReLU(inplace=True)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (9): ReLU(inplace=True)
  (10): ResnetBlock(
    (conv_block): Sequential(
      (0): ReflectionPad2d((1, 1, 1, 1))
      (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
      (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): ReLU(inplace=True)
      (4): ReflectionPad2d((1, 1, 1, 1))
      (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    

In [8]:
#export
def ConvNLReLU(i, o, norm, k=3, s=1, p=1, b=True, act=True, slope=0.2, init=nn.init.kaiming_normal_):
    '''Convolutional layer + normalization layer + ReLU activation.
        i: channel in
        o: channel out
        norm: normalization layer
        k: kernel size
        s: stride size
        p: padding size
        b: bias or not in TransposeConv
        act: whether to append ReLU after PadConvN
        slope: leaky relu slope
        init: initialization function
    '''
    layers = []
    #conv
    conv = nn.Conv2d(i, o, k, s, p, bias=b)
    if init:
        init(conv.weight)
        if hasattr(conv, 'bias') and hasattr(conv.bias, 'data'):
            conv.bias.data.fill_(0.)
    layers.append(conv)
    #norm
    if norm: layers.append(norm(o))
    #activation
    if act:  layers.append(nn.LeakyReLU(slope, inplace=True))
    return layers

In [9]:
#export
def discriminator(i, c=64, norm=None, n_layer=3, act=False):
    '''Discriminator of GAN.
        i: channel in
        c: start channel (changes as we build the discriminator blocks)
        norm: normalization layer
        act: whether to append activation at the end of discriminator
    '''
    layers = []
    norm = norm if norm else nn.InstanceNorm2d
    b = norm == nn.InstanceNorm2d
    layers += ConvNLReLU(i, c, None, 4, 2, 1)
    for i in range(n_layer-1):
        new_c = c * 2 if i <= 3 else c
        layers += ConvNLReLU(c, new_c, norm, 4, 2, 1, b)
        c = new_c
    new_c = c * 2 if i <= 3 else c
    layers += ConvNLReLU(c, new_c, norm, 4, 1, 1, b)
    layers.append(nn.Conv2d(new_c, 1, 4, 1, 1))
    if act: layers.append(nn.Sigmoid())
    return nn.Sequential(*layers)

In [10]:
print(discriminator(3))

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2, inplace=True)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (4): LeakyReLU(negative_slope=0.2, inplace=True)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (7): LeakyReLU(negative_slope=0.2, inplace=True)
  (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (10): LeakyReLU(negative_slope=0.2, inplace=True)
  (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
)


In [11]:
#export
class CycleGAN(nn.Module):
    def __init__(self, i, o, c=64, norm=None, gen_layers=6, dis_layers=3, dropout=0., lsgan=True):
        '''Cycle GAN with two generators and discriminators.
            i: channel in
            o: channel out
            c: start channel (dynamic)
            norm: normalization layer
            gen_layers: number of generator blocks
            dis_layer: number of discriminator blocks
            dropout: dropout factor (0 to 1)
            lsgan: boolean factor for least square gan (https://arxiv.org/abs/1611.04076)
        '''
        super().__init__()
        self.G1 = generator(i, o, c, norm, dropout, gen_layers) # domain 2 to 1
        self.G2 = generator(i, o, c, norm, dropout, gen_layers) # domain 1 to 2
        self.D1 = discriminator(i, c, norm, dis_layers, act=not lsgan) # test domain 1
        self.D2 = discriminator(i, c, norm, dis_layers, act=not lsgan) # test domain 2
    
    def forward(self, real1, real2):
        fake1, fake2 = self.G1(real2), self.G2(real1)
        if not self.training:
            return torch.cat([fake_A.unsqueeze(-1), fake_B.unsqueeze(-1)], -1)
        iden1, iden2 = self.G1(real1), self.G2(real2)
        return [fake1, fake2, iden1, iden2]

In [12]:
#export
class AdaptiveLoss(nn.Module):
    def __init__(self, critic):
        '''Critic wrapper for creating binary target tensors for loss computation.
            critic: loss module
        '''
        super().__init__()
        self.critic = critic
    
    def forward(self, out, tar, **kwargs):
        target = out.new_ones(*output.size()) if tar else out.new_zeros(*output.size())
        return self.critic(out, target, **kwargs)

In [13]:
#export
class CycleGANLoss(nn.Module):
    def __init__(self, cycleGAN, w1=10., w2=10., wi=0.5, lsgan=True):
        '''Cycle GAN loss layer with identity loss, generator loss, and discriminator loss.
            w1: lambda factor for loss in domain 1
            w2: lambda factor for loss in domain 2
            wi: lambda factor for identity loss
            lsgan: factor for https://arxiv.org/abs/1611.04076
        '''
        super().__init__()
        self.cycleGAN, self.w1, self.w2, self.wi = cycleGAN, w1, w2, wi
        self.critic = AdaptiveLoss(F.mse_loss if lsgan else F.binary_cross_entropy)
    
    def set_input(self, inp):
        self.real1, self.real2 = inp
    
    def forward(self, out, tar):
        fake1, fake2, iden1, iden2 = out
        iden_loss1 = self.lambdaI * (self.lambda1 * F.l1_loss(iden1, self.real1))
        iden_loss2 = self.lambdaI * (self.lambda2 * F.l1_loss(iden2, self.real2))
        self.iden_loss = iden_loss1 + iden_loss2
        
        gen_loss1 = self.critic(self.cycleGAN.D1(fake1), True)
        gen_loss2 = self.critic(self.cycleGAN.D2(fake2), True)
        self.gen_loss = gen_loss1 + gen_loss2
        
        cycle_loss1  = self.lambda1 * F.l1_loss(self.cycleGAN.G1(fake2), self.real1)
        cycle_loss2  = self.lambda2 * F.l1_loss(self.cycleGAN.G2(fake1), self.real2)
        self.cycle_loss = cycle_loss1 + cycle_loss2
        
        return self.iden_loss + self.gen_loss + self.cycle_loss

In [14]:
#export
class CycleGANTrainer(LearnerCallback):
    def __init__(self):
        '''Callback for the special training procedure of Cycle GAN.'''
        super().__init__()
        
    def _set_trainable(self, D1=False, D2=False):
        gen = (not D1) and (not D2)
        requires_grad(self.learn.model.G1, gen)
        requires_grad(self.learn.model.G2, gen)
        requires_grad(self.learn.model.D1, D1)
        requires_grad(self.learn.model.D2, D2)
        if not gen:
            self.opt_D1.lr, self.opt_D1.mom = self.learn.opt.lr, self.learn.opt.mom
            self.opt_D1.wd, self.opt_D1.beta = self.learn.opt.wd, self.learn.opt.beta
            self.opt_D2.lr, self.opt_D2.mom = self.learn.opt.lr, self.learn.opt.mom
            self.opt_D2.wd, self.opt_D2.beta = self.learn.opt.wd, self.learn.opt.beta
    
    def before_fit(self, **kwargs):
        self.G1, self.G2 = self.learn.model.G1, self.learn.model.G2
        self.D1, self.D2 = self.learn.model.D1, self.learn.model.D2
        self.crit = self.learn.loss_func.crit
        if not getattr(self,'opt_G',None):
            self.opt_G = self.learn.opt.new([nn.Sequential(*flatten_model(self.G1), *flatten_model(self.G2))])
        else: 
            self.opt_G.lr, self.opt_G.wd = self.opt.lr, self.opt.wd
            self.opt_G.mom, self.opt_G.beta = self.opt.mom, self.opt.beta
        if not getattr(self,'opt_D1',None):
            self.opt_D1 = self.learn.opt.new([nn.Sequential(*flatten_model(self.D1))])
        if not getattr(self,'opt_D2',None):
            self.opt_D2 = self.learn.opt.new([nn.Sequential(*flatten_model(self.D2))])
        self.learn.opt.opt = self.opt_G.opt
        self._set_trainable()
        self.id_smter, self.gen_smter, self.cyc_smter = SmoothenValue(0.98), SmoothenValue(0.98), SmoothenValue(0.98)
        self.da_smter, self.db_smter = SmoothenValue(0.98), SmoothenValue(0.98)
        self.recorder.add_metric_names(['id_loss', 'gen_loss', 'cyc_loss', 'D1_loss', 'D2_loss'])
        
    def before_batch(self, last_input, **kwargs):
        self.learn.loss_func.set_input(last_input)
    
    def before_loss_back(self, **kwargs):
        self.id_smter.add_value(self.loss_func.id_loss.detach().cpu())
        self.gen_smter.add_value(self.loss_func.gen_loss.detach().cpu())
        self.cyc_smter.add_value(self.loss_func.cyc_loss.detach().cpu())
    
    def after_batch(self, last_input, last_output, **kwargs):
        self.G1.zero_grad(); self.G2.zero_grad()
        fake1, fake2 = last_output[0].detach(), last_output[1].detach()
        real1, real2 = last_input
        self._set_trainable(D1=True)
        self.D1.zero_grad()
        loss_D1 = 0.5 * (self.crit(self.D1(real1), True) + self.crit(self.D1(fake1), False))
        self.da_smter.add_value(loss_D1.detach().cpu())
        loss_D1.backward()
        self.opt_D1.step()
        self._set_trainable(D2=True)
        self.D2.zero_grad()
        loss_D2 = 0.5 * (self.crit(self.D2(real2), True) + self.crit(self.D2(fake2), False))
        self.db_smter.add_value(loss_D2.detach().cpu())
        loss_D2.backward()
        self.opt_D2.step()
        self._set_trainable()
        
    def after_epoch(self, last_metrics, **kwargs):
        return add_metrics(last_metrics, [s.smooth for s in [self.id_smter, self.gen_smter, self.cyc_smter, self.da_smter,self.db_smter]])

In [15]:
cycle_gan = CycleGAN(3, 3, gen_layers=9)
cycle_gan

CycleGAN(
  (G1): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): Co